In [1]:
import torch
import torch.nn as nn
import os
import wandb
import random
import argparse
import numpy as np
from tqdm import tqdm
from transformers import BertModel
from transformers import AdamW

/mnt/cfs/anaconda3/envs/pykt-hzw3.8/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 数据预处理部分

In [2]:
from torch.utils.data import Dataset
import json

class ErrorDetectionDataset(Dataset):
    def __init__(
            self,
            data_path,
            coarse_labels={
                "字符级错误": 0,
                "成分残缺型错误": 1, 
                "成分冗余型错误": 2,
                "成分搭配不当型错误": 3
            },
            fine_labels={
                "缺字漏字": 0,
                "错别字错误": 1,
                "缺少标点": 2,
                "错用标点": 3,
                "主语不明": 4,
                "谓语残缺": 5,
                "宾语残缺": 6,
                "其他成分残缺": 7,
                "主语多余": 8,
                "谓词多余": 9,
                "其他成分多余": 10,
                "语序不当": 11,
                "动宾搭配不当": 12,
                "其他搭配不当": 13
            }
    ):
        self.data_path = data_path
        self.coarse_labels = coarse_labels
        self.fine_labels = fine_labels
        self._get_data()
    
    def _get_data(self):
        with open(self.data_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        self.data = []
        for item in data:
            sent = item['sent']
            coarse_types = item['CourseGrainedErrorType']
            fine_types = item['FineGrainedErrorType']
            
            # 构建粗粒度标签（多标签）
            coarse_label = [0] * len(self.coarse_labels)
            for c_type in coarse_types:
                if c_type in self.coarse_labels:
                    coarse_label[self.coarse_labels[c_type]] = 1
            
            # 构建细粒度标签（多标签）
            fine_label = [0] * len(self.fine_labels)
            for f_type in fine_types:
                if f_type in self.fine_labels:
                    fine_label[self.fine_labels[f_type]] = 1
            
            self.data.append((sent, coarse_label, fine_label, item.get('sent_id', -1)))
    
    def __len__(self):
        return len(self.data)
    
    def get_coarse_labels(self):
        return self.coarse_labels
    
    def get_fine_labels(self):
        return self.fine_labels
    
    def __getitem__(self, idx):
        return self.data[idx]

# 数据加载部分

In [3]:
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer

class ErrorDetectionDataLoader:
    def __init__(
        self,
        dataset,
        batch_size=16,
        max_length=128,
        shuffle=True,
        drop_last=True,
        device=None,
        tokenizer_name='bge-large-zh-v1.5'
    ):
        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
        self.dataset = dataset
        self.batch_size = batch_size
        self.max_length = max_length
        self.shuffle = shuffle
        self.drop_last = drop_last
        
        if device is None:
            self.device = torch.device(
                'cuda' if torch.cuda.is_available() else 'cpu'
            )
        else:
            self.device = device
        
        self.loader = DataLoader(
            dataset=self.dataset,
            batch_size=self.batch_size,
            collate_fn=self.collate_fn,
            shuffle=self.shuffle,
            drop_last=self.drop_last
        )
    
    def collate_fn(self, data):
        sents = [item[0] for item in data]
        coarse_labels = [item[1] for item in data]
        fine_labels = [item[2] for item in data]
        sent_ids = [item[3] for item in data]
        
        # 编码文本
        encoded = self.tokenizer.batch_encode_plus(
            batch_text_or_text_pairs=sents,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt',
            return_length=True
        )
        
        input_ids = encoded['input_ids'].to(self.device)
        attention_mask = encoded['attention_mask'].to(self.device)
        token_type_ids = encoded.get('token_type_ids', None)
        
        if token_type_ids is not None:
            token_type_ids = token_type_ids.to(self.device)
        
        # 处理标签
        if coarse_labels[0] == -1:
            coarse_labels = None
            fine_labels = None
        else:
            coarse_labels = torch.tensor(coarse_labels, dtype=torch.float).to(self.device)
            fine_labels = torch.tensor(fine_labels, dtype=torch.float).to(self.device)
        
        return input_ids, attention_mask, token_type_ids, coarse_labels, fine_labels, sent_ids
    
    def __iter__(self):
        for data in self.loader:
            yield data
    
    def __len__(self):
        return len(self.loader)

 # 模型部分

In [4]:
import torch
import torch.nn as nn
from transformers import AutoModel

class HierarchicalErrorClassifier(nn.Module):
    def __init__(
        self, 
        pretrained_model_name, 
        num_coarse_labels=4, 
        num_fine_labels=14, 
        freeze_pooler=False, 
        dropout=0.2
    ):
        super().__init__()
        
        self.freeze_pooler = freeze_pooler
        self.bert = AutoModel.from_pretrained(pretrained_model_name)
        self.dropout = nn.Dropout(dropout) if dropout > 0 else nn.Identity()
        
        if freeze_pooler:
            for param in self.bert.pooler.parameters():
                param.requires_grad = False
        
        # 粗粒度分类器
        self.coarse_classifier = nn.Linear(self.bert.config.hidden_size, num_coarse_labels)
        
        # 细粒度分类器
        self.fine_classifier = nn.Linear(self.bert.config.hidden_size, num_fine_labels)
        
        # 定义粗粒度类别和对应的细粒度索引的映射
        self.coarse_to_fine_indices = {
            0: [0, 1, 2, 3],    # 字符级错误
            1: [4, 5, 6, 7],    # 成分残缺型错误
            2: [8, 9, 10],      # 成分冗余型错误
            3: [11, 12, 13]     # 成分搭配不当型错误
        }
    
    def forward(self, input_ids, attention_mask, token_type_ids=None):
        # BERT编码
        if token_type_ids is not None:
            outputs = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        else:
            outputs = self.bert(input_ids, attention_mask=attention_mask)
        
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)
        
        # 粗粒度分类
        coarse_logits = self.coarse_classifier(pooled_output)
        coarse_probs = torch.sigmoid(coarse_logits)
        
        # 细粒度分类
        fine_logits = self.fine_classifier(pooled_output)
        fine_probs = torch.sigmoid(fine_logits)
        
        return coarse_probs, fine_probs
    
    def apply_hierarchical_constraint(self, coarse_preds, fine_preds):
        """
        应用层次约束：如果粗粒度类别预测为负，则该粗粒度下的所有细粒度类别均设为负
        
        Args:
            coarse_preds: 粗粒度预测结果，shape [batch_size, num_coarse_labels]
            fine_preds: 细粒度预测结果，shape [batch_size, num_fine_labels]
            
        Returns:
            应用约束后的细粒度预测结果
        """
        constrained_fine_preds = fine_preds.clone()
        
        # 遍历每个样本
        for i in range(coarse_preds.size(0)):
            # 对每个粗粒度类别
            for coarse_idx, fine_indices in self.coarse_to_fine_indices.items():
                # 如果粗粒度为负，则对应的细粒度全部设为负
                if coarse_preds[i, coarse_idx] == 0:
                    constrained_fine_preds[i, fine_indices] = 0
        
        return constrained_fine_preds

# 训练、验证和测试部分

In [ ]:
import os
import wandb
import random
import argparse
from tqdm import tqdm

import torch
import torch.nn as nn
import numpy as np
from transformers import AdamW
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

# 导入自定义模块
# from error_detection_dataset import ErrorDetectionDataset
# from error_detection_dataloader import ErrorDetectionDataLoader
# from hierarchical_classifier_model import HierarchicalErrorClassifier

def argparser():
    parser = argparse.ArgumentParser()
    parser.add_argument('--model_name', type=str, default='./models/bge-large-zh-v1.5')
    parser.add_argument('--num_coarse_labels', type=int, default=4)
    parser.add_argument('--num_fine_labels', type=int, default=14)
    parser.add_argument('--dropout', type=float, default=0.2)
    parser.add_argument('--freeze_pooler', action='store_true', help='Flag to freeze the pooler layer')
    parser.add_argument('--batch_size', type=int, default=16)
    parser.add_argument('--max_length', type=int, default=128)
    parser.add_argument('--lr', type=float, default=1e-5)
    parser.add_argument('--epochs', type=int, default=40)
    parser.add_argument('--device', type=str, required=False)
    parser.add_argument('--project', type=str, default='hierarchical_error_detection')
    parser.add_argument('--entity', type=str, default='akccc')
    parser.add_argument('--name', type=str, required=False)
    parser.add_argument('--seed', type=int, default=42)
    parser.add_argument('--data_path', type=str, default='../datas/train.json')
    parser.add_argument('--val_data_path', type=str, default='../datas/val.json')
    parser.add_argument('--test_data_path', type=str, default='../dats/val.json')
    parser.add_argument('--checkpoint_dir', type=str, default='checkpoints')
    parser.add_argument('--threshold', type=float, default=0.5)
    parser.add_argument('--patience', type=int, default=5)
    
    return parser.parse_args()



# 如果在Jupyter Notebook中运行，可以使用这个自定义参数函数替代argparser
def get_default_configs():
    """在Jupyter环境中使用的默认配置，避免argparse解析错误"""
    class Args:
        def __init__(self):
            self.model_name = './models/bge-large-zh-v1.5'
            self.num_coarse_labels = 4
            self.num_fine_labels = 14
            self.dropout = 0.2
            self.freeze_pooler = False
            self.batch_size = 16
            self.max_length = 128
            self.lr = 1e-5
            self.epochs = 20
            self.device = None
            self.project = 'hierarchical_error_detection'
            self.entity = 'akccc'
            self.name = None
            self.seed = 42
            self.data_path = '../datas/train.json'
            self.val_data_path = '../datas/val.json'
            self.test_data_path = '../datas/val.json'
            self.checkpoint_dir = 'checkpoints'
            self.threshold = 0.5
            self.patience = 5
            self.exp_name = 'default_run'
    return Args()




def calculate_metrics(labels, predictions, average='micro'):
    """
    计算各种评估指标
    
    Args:
        labels: 真实标签
        predictions: 预测标签
        average: 平均方法，'micro'或'macro'
        
    Returns:
        包含各种指标的字典
    """
    # 将数组转换为numpy格式以确保兼容性
    labels = np.array(labels)
    predictions = np.array(predictions)
    
    # 计算微平均和宏平均的F1分数
    micro_f1 = f1_score(labels, predictions, average='micro')
    macro_f1 = f1_score(labels, predictions, average='macro')
    
    # 计算样本级别的准确率（每个样本的所有标签都要正确）
    sample_acc = accuracy_score(labels, predictions)
    
    return {
        'micro_f1': micro_f1 * 100,  # 转换为百分比
        'macro_f1': macro_f1 * 100,
        'accuracy': sample_acc * 100
    }

def train(configs):
    # 初始化wandb
    wandb.init(
        project=configs.project,
        entity=configs.entity,
        name=configs.exp_name,
    )

    # 配置wandb
    wandb_config = wandb.config
    wandb_config.model_name = configs.model_name
    wandb_config.num_coarse_labels = configs.num_coarse_labels
    wandb_config.num_fine_labels = configs.num_fine_labels
    wandb_config.dropout = configs.dropout
    wandb_config.freeze_pooler = configs.freeze_pooler
    wandb_config.batch_size = configs.batch_size
    wandb_config.max_length = configs.max_length
    wandb_config.lr = configs.lr
    wandb_config.epochs = configs.epochs
    wandb_config.device = configs.device
    wandb_config.seed = configs.seed
    wandb_config.threshold = configs.threshold

    # 设置随机种子
    random.seed(configs.seed)
    np.random.seed(configs.seed)
    torch.manual_seed(configs.seed)
    
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
    # 创建检查点目录
    checkpoint_dir = os.path.join(configs.checkpoint_dir, configs.exp_name)
    os.makedirs(checkpoint_dir, exist_ok=True)
    
    # 加载数据集
    train_dataset = ErrorDetectionDataset(configs.data_path)
    val_dataset = ErrorDetectionDataset(configs.val_data_path)
    test_dataset = ErrorDetectionDataset(configs.test_data_path)

    # 创建数据加载器
    train_dataloader = ErrorDetectionDataLoader(
        dataset=train_dataset,
        batch_size=configs.batch_size,
        max_length=configs.max_length,
        shuffle=True,
        drop_last=True,
        device=configs.device,
        tokenizer_name=configs.model_name
    )

    val_dataloader = ErrorDetectionDataLoader(
        dataset=val_dataset,
        batch_size=configs.batch_size,
        max_length=configs.max_length,
        shuffle=False,
        drop_last=False,
        device=configs.device,
        tokenizer_name=configs.model_name
    )

    test_dataloader = ErrorDetectionDataLoader(
        dataset=test_dataset,
        batch_size=configs.batch_size,
        max_length=configs.max_length,
        shuffle=False,
        drop_last=False,
        device=configs.device,
        tokenizer_name=configs.model_name
    )

    # 创建模型
    model = HierarchicalErrorClassifier(
        pretrained_model_name=configs.model_name,
        num_coarse_labels=configs.num_coarse_labels,
        num_fine_labels=configs.num_fine_labels,
        dropout=configs.dropout,
        freeze_pooler=configs.freeze_pooler
    ).to(configs.device)

    # 定义损失函数
    criterion = nn.BCELoss()

    # 定义优化器
    optimizer = AdamW(
        filter(lambda p: p.requires_grad, model.parameters()),
        lr=configs.lr
    )

    # 初始化最佳验证损失和早停计数器
    best_val_f1 = 0
    patience_counter = 0
    
    # 监控模型
    wandb.watch(model, log='all')
    
    # 训练循环
    for epoch in range(configs.epochs):
        # 训练阶段
        model.train()
        train_loss = 0.0
        all_coarse_preds = []
        all_coarse_labels = []
        all_fine_preds = []
        all_fine_labels = []
        all_constrained_fine_preds = []
        
        with tqdm(
            train_dataloader,
            total=len(train_dataloader),
            desc=f'Epoch {epoch + 1}/{configs.epochs}',
            unit='batch',
            ncols=100
        ) as pbar:
            for input_ids, attention_mask, token_type_ids, coarse_labels, fine_labels, _ in pbar:
                optimizer.zero_grad()
                
                # 前向传播
                coarse_probs, fine_probs = model(input_ids, attention_mask, token_type_ids)
                
                # 计算损失
                coarse_loss = criterion(coarse_probs, coarse_labels)
                fine_loss = criterion(fine_probs, fine_labels)
                loss = coarse_loss + fine_loss
                
                # 反向传播
                loss.backward()
                optimizer.step()
                
                train_loss += loss.item()
                
                # 收集预测结果
                coarse_preds = (coarse_probs > configs.threshold).float().cpu().numpy()
                fine_preds = (fine_probs > configs.threshold).float().cpu().numpy()
                constrained_fine_preds = model.apply_hierarchical_constraint(
                    (coarse_probs > configs.threshold).float(), 
                    (fine_probs > configs.threshold).float()
                ).cpu().numpy()
                
                all_coarse_preds.extend(coarse_preds)
                all_coarse_labels.extend(coarse_labels.cpu().numpy())
                all_fine_preds.extend(fine_preds)
                all_fine_labels.extend(fine_labels.cpu().numpy())
                all_constrained_fine_preds.extend(constrained_fine_preds)
                
                # 更新进度条
                pbar.set_postfix(
                    loss=f'{loss.item():.3f}',
                    coarse_loss=f'{coarse_loss.item():.3f}',
                    fine_loss=f'{fine_loss.item():.3f}'
                )
                
                # 记录到wandb
                wandb.log({
                    'batch_loss': loss.item(),
                    'batch_coarse_loss': coarse_loss.item(),
                    'batch_fine_loss': fine_loss.item()
                })
        
        # 计算训练指标
        train_loss = train_loss / len(train_dataloader)
        
        # 计算各种评估指标
        train_coarse_metrics_micro = calculate_metrics(all_coarse_labels, all_coarse_preds, average='micro')
        train_coarse_metrics_macro = calculate_metrics(all_coarse_labels, all_coarse_preds, average='macro')
        train_fine_metrics_micro = calculate_metrics(all_fine_labels, all_fine_preds, average='micro')
        train_fine_metrics_macro = calculate_metrics(all_fine_labels, all_fine_preds, average='macro')
        train_constrained_fine_metrics_micro = calculate_metrics(all_fine_labels, all_constrained_fine_preds, average='micro')
        train_constrained_fine_metrics_macro = calculate_metrics(all_fine_labels, all_constrained_fine_preds, average='macro')
        
        # 保存模型检查点
        checkpoint_path = os.path.join(checkpoint_dir, f'epoch_{epoch + 1}.pt')
        torch.save(model.state_dict(), checkpoint_path)
        wandb.save(checkpoint_path)
        
        # 验证阶段
        model.eval()
        val_loss = 0.0
        all_coarse_preds = []
        all_coarse_labels = []
        all_fine_preds = []
        all_fine_labels = []
        all_constrained_fine_preds = []
        
        with torch.no_grad():
            for input_ids, attention_mask, token_type_ids, coarse_labels, fine_labels, _ in val_dataloader:
                # 前向传播
                coarse_probs, fine_probs = model(input_ids, attention_mask, token_type_ids)
                
                # 应用层次约束
                coarse_preds = (coarse_probs > configs.threshold).float()
                fine_preds = (fine_probs > configs.threshold).float()
                constrained_fine_preds = model.apply_hierarchical_constraint(coarse_preds, fine_preds)
                
                # 计算损失
                coarse_loss = criterion(coarse_probs, coarse_labels)
                fine_loss = criterion(fine_probs, fine_labels)
                loss = coarse_loss + fine_loss
                
                val_loss += loss.item()
                
                # 收集预测结果
                all_coarse_preds.extend(coarse_preds.cpu().numpy())
                all_coarse_labels.extend(coarse_labels.cpu().numpy())
                all_fine_preds.extend(fine_preds.cpu().numpy())
                all_constrained_fine_preds.extend(constrained_fine_preds.cpu().numpy())
                all_fine_labels.extend(fine_labels.cpu().numpy())
        
        # 计算验证指标
        val_loss = val_loss / len(val_dataloader)
        
        # 计算各种评估指标
        val_coarse_metrics_micro = calculate_metrics(all_coarse_labels, all_coarse_preds, average='micro')
        val_coarse_metrics_macro = calculate_metrics(all_coarse_labels, all_coarse_preds, average='macro')
        val_fine_metrics_micro = calculate_metrics(all_fine_labels, all_fine_preds, average='micro')
        val_fine_metrics_macro = calculate_metrics(all_fine_labels, all_fine_preds, average='macro')
        val_constrained_fine_metrics_micro = calculate_metrics(all_fine_labels, all_constrained_fine_preds, average='micro')
        val_constrained_fine_metrics_macro = calculate_metrics(all_fine_labels, all_constrained_fine_preds, average='macro')
        
        # 输出训练和验证指标
        print(f'\nEpoch {epoch+1}/{configs.epochs}:')
        print(f'  Train Loss: {train_loss:.4f}')
        print(f'  Train Coarse-grained Metrics:')
        print(f'    Micro F1: {train_coarse_metrics_micro["micro_f1"]:.2f}')
        print(f'    Macro F1: {train_coarse_metrics_macro["macro_f1"]:.2f}')
        print(f'    Accuracy: {train_coarse_metrics_micro["accuracy"]:.2f}')
        print(f'  Train Fine-grained Metrics (Unconstrained):')
        print(f'    Micro F1: {train_fine_metrics_micro["micro_f1"]:.2f}')
        print(f'    Macro F1: {train_fine_metrics_macro["macro_f1"]:.2f}')
        print(f'    Accuracy: {train_fine_metrics_micro["accuracy"]:.2f}')
        print(f'  Train Fine-grained Metrics (Constrained):')
        print(f'    Micro F1: {train_constrained_fine_metrics_micro["micro_f1"]:.2f}')
        print(f'    Macro F1: {train_constrained_fine_metrics_macro["macro_f1"]:.2f}')
        print(f'    Accuracy: {train_constrained_fine_metrics_micro["accuracy"]:.2f}')
        
        print(f'  Val Loss: {val_loss:.4f}')
        print(f'  Val Coarse-grained Metrics:')
        print(f'    Micro F1: {val_coarse_metrics_micro["micro_f1"]:.2f}')
        print(f'    Macro F1: {val_coarse_metrics_macro["macro_f1"]:.2f}')
        print(f'    Accuracy: {val_coarse_metrics_micro["accuracy"]:.2f}')
        print(f'  Val Fine-grained Metrics (Unconstrained):')
        print(f'    Micro F1: {val_fine_metrics_micro["micro_f1"]:.2f}')
        print(f'    Macro F1: {val_fine_metrics_macro["macro_f1"]:.2f}')
        print(f'    Accuracy: {val_fine_metrics_micro["accuracy"]:.2f}')
        print(f'  Val Fine-grained Metrics (Constrained):')
        print(f'    Micro F1: {val_constrained_fine_metrics_micro["micro_f1"]:.2f}')
        print(f'    Macro F1: {val_constrained_fine_metrics_macro["macro_f1"]:.2f}')
        print(f'    Accuracy: {val_constrained_fine_metrics_micro["accuracy"]:.2f}')
        
        # 记录到wandb
        wandb.log({
            'train_loss': train_loss,
            'train_coarse_micro_f1': train_coarse_metrics_micro["micro_f1"],
            'train_coarse_macro_f1': train_coarse_metrics_macro["macro_f1"],
            'train_coarse_accuracy': train_coarse_metrics_micro["accuracy"],
            'train_fine_micro_f1': train_fine_metrics_micro["micro_f1"],
            'train_fine_macro_f1': train_fine_metrics_macro["macro_f1"],
            'train_fine_accuracy': train_fine_metrics_micro["accuracy"],
            'train_constrained_fine_micro_f1': train_constrained_fine_metrics_micro["micro_f1"],
            'train_constrained_fine_macro_f1': train_constrained_fine_metrics_macro["macro_f1"],
            'train_constrained_fine_accuracy': train_constrained_fine_metrics_micro["accuracy"],
            'val_loss': val_loss,
            'val_coarse_micro_f1': val_coarse_metrics_micro["micro_f1"],
            'val_coarse_macro_f1': val_coarse_metrics_macro["macro_f1"],
            'val_coarse_accuracy': val_coarse_metrics_micro["accuracy"],
            'val_fine_micro_f1': val_fine_metrics_micro["micro_f1"],
            'val_fine_macro_f1': val_fine_metrics_macro["macro_f1"],
            'val_fine_accuracy': val_fine_metrics_micro["accuracy"],
            'val_constrained_fine_micro_f1': val_constrained_fine_metrics_micro["micro_f1"],
            'val_constrained_fine_macro_f1': val_constrained_fine_metrics_macro["macro_f1"],
            'val_constrained_fine_accuracy': val_constrained_fine_metrics_micro["accuracy"],
            'epoch': epoch + 1
        })
        
        # 检查是否保存最佳模型并应用早停
        if val_constrained_fine_metrics_micro["micro_f1"] > best_val_f1:
            best_val_f1 = val_constrained_fine_metrics_micro["micro_f1"]
            torch.save(model.state_dict(), os.path.join(checkpoint_dir, 'best_model.pt'))
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= configs.patience:
                print('Early stopping triggered.')
                break
    
    # 加载最佳模型进行测试
    print("\n===== Testing best model =====")
    model.load_state_dict(torch.load(os.path.join(checkpoint_dir, 'best_model.pt')))
    model.eval()
    
    all_coarse_preds = []
    all_coarse_labels = []
    all_fine_preds = []
    all_fine_labels = []
    all_constrained_fine_preds = []
    
    with torch.no_grad():
        for input_ids, attention_mask, token_type_ids, coarse_labels, fine_labels, _ in test_dataloader:
            # 前向传播
            coarse_probs, fine_probs = model(input_ids, attention_mask, token_type_ids)
            
            # 应用层次约束
            coarse_preds = (coarse_probs > configs.threshold).float()
            fine_preds = (fine_probs > configs.threshold).float()
            constrained_fine_preds = model.apply_hierarchical_constraint(coarse_preds, fine_preds)
            
            # 收集预测结果
            all_coarse_preds.extend(coarse_preds.cpu().numpy())
            all_coarse_labels.extend(coarse_labels.cpu().numpy())
            all_fine_preds.extend(fine_preds.cpu().numpy())
            all_constrained_fine_preds.extend(constrained_fine_preds.cpu().numpy())
            all_fine_labels.extend(fine_labels.cpu().numpy())
    
    # 计算测试指标
    test_coarse_metrics_micro = calculate_metrics(all_coarse_labels, all_coarse_preds, average='micro')
    test_coarse_metrics_macro = calculate_metrics(all_coarse_labels, all_coarse_preds, average='macro')
    test_fine_metrics_micro = calculate_metrics(all_fine_labels, all_fine_preds, average='micro')
    test_fine_metrics_macro = calculate_metrics(all_fine_labels, all_fine_preds, average='macro')
    test_constrained_fine_metrics_micro = calculate_metrics(all_fine_labels, all_constrained_fine_preds, average='micro')
    test_constrained_fine_metrics_macro = calculate_metrics(all_fine_labels, all_constrained_fine_preds, average='macro')
    
    # 输出测试结果
    print("\n===== Final Test Results =====")
    print(f"Final micro f1: {test_constrained_fine_metrics_micro['micro_f1']:.2f}")
    print(f"Final macro f1: {test_constrained_fine_metrics_macro['macro_f1']:.2f}")
    
    print("\nCoarse-grained micro f1: {:.2f}".format(test_coarse_metrics_micro['micro_f1']))
    print("Fine-grained micro f1: {:.2f}".format(test_constrained_fine_metrics_micro['micro_f1']))
    
    print("\nCoarse-grained macro f1: {:.2f}".format(test_coarse_metrics_macro['macro_f1']))
    print("Fine-grained macro f1: {:.2f}".format(test_constrained_fine_metrics_macro['macro_f1']))
    
    print("\nAccuracy: {:.2f}".format(test_constrained_fine_metrics_micro['accuracy']))
    
    # 以表格形式输出所有指标（与给定的评估表格格式一致）
    print("\n")
    print("+" + "-"*15 + "+" + "-"*15 + "+" + "-"*15 + "+" + "-"*15 + "+" + "-"*15 + "+" + "-"*15 + "+")
    print("| {:<13} | {:<13} | {:<13} | {:<13} | {:<13} | {:<13} |".format(
        "Final", "Final", "Course-", "Fine-grained", "Course-", "Fine-grained"))
    print("| {:<13} | {:<13} | {:<13} | {:<13} | {:<13} | {:<13} |".format(
        "micro f1", "macro f1", "grained micro f1", "micro f1", "grained macro f1", "macro f1"))
    print("+" + "-"*15 + "+" + "-"*15 + "+" + "-"*15 + "+" + "-"*15 + "+" + "-"*15 + "+" + "-"*15 + "+")
    print("| {:<13.2f} | {:<13.2f} | {:<13.2f} | {:<13.2f} | {:<13.2f} | {:<13.2f} |".format(
        test_constrained_fine_metrics_micro['micro_f1'],
        test_constrained_fine_metrics_macro['macro_f1'],
        test_coarse_metrics_micro['micro_f1'],
        test_constrained_fine_metrics_micro['micro_f1'],
        test_coarse_metrics_macro['macro_f1'],
        test_constrained_fine_metrics_macro['macro_f1']
    ))
    print("+" + "-"*15 + "+" + "-"*15 + "+" + "-"*15 + "+" + "-"*15 + "+" + "-"*15 + "+" + "-"*15 + "+")
    
    # 记录最终结果到wandb
    wandb.log({
        'test_coarse_micro_f1': test_coarse_metrics_micro["micro_f1"],
        'test_coarse_macro_f1': test_coarse_metrics_macro["macro_f1"],
        'test_coarse_accuracy': test_coarse_metrics_micro["accuracy"],
        'test_fine_micro_f1': test_fine_metrics_micro["micro_f1"],
        'test_fine_macro_f1': test_fine_metrics_macro["macro_f1"],
        'test_fine_accuracy': test_fine_metrics_micro["accuracy"],
        'test_constrained_fine_micro_f1': test_constrained_fine_metrics_micro["micro_f1"],
        'test_constrained_fine_macro_f1': test_constrained_fine_metrics_macro["macro_f1"],
        'test_constrained_fine_accuracy': test_constrained_fine_metrics_micro["accuracy"],
        'final_micro_f1': test_constrained_fine_metrics_micro["micro_f1"],
        'final_macro_f1': test_constrained_fine_metrics_macro["macro_f1"]
    })
    
    # 完成wandb记录
    wandb.finish()

def predict(model, text, tokenizer, device, threshold=0.5):
    """
    对单个文本进行预测
    """
    model.eval()
    
    # 编码文本
    encoded = tokenizer(
        text,
        truncation=True,
        padding='max_length',
        max_length=128,
        return_tensors='pt'
    )
    
    input_ids = encoded['input_ids'].to(device)
    attention_mask = encoded['attention_mask'].to(device)
    token_type_ids = encoded.get('token_type_ids', None)
    
    if token_type_ids is not None:
        token_type_ids = token_type_ids.to(device)
    
    # 获取预测结果
    with torch.no_grad():
        coarse_probs, fine_probs = model(input_ids, attention_mask, token_type_ids)
        
        # 应用阈值
        coarse_preds = (coarse_probs > threshold).float()
        fine_preds = (fine_probs > threshold).float()
        
        # 应用层次约束
        constrained_fine_preds = model.apply_hierarchical_constraint(coarse_preds, fine_preds)
    
    # 映射结果到标签
    coarse_indices = torch.nonzero(coarse_preds[0]).cpu().numpy().flatten()
    fine_indices = torch.nonzero(constrained_fine_preds[0]).cpu().numpy().flatten()
    
    # 将索引转换为标签名称（需要模型中有这些映射）
    coarse_label_map = {v: k for k, v in model.coarse_labels.items()}
    fine_label_map = {v: k for k, v in model.fine_labels.items()}
    
    predicted_coarse = [coarse_label_map[idx] for idx in coarse_indices]
    predicted_fine = [fine_label_map[idx] for idx in fine_indices]
    
    return predicted_coarse, predicted_fine




# 在以下主函数中添加判断Jupyter环境的逻辑
if __name__ == '__main__':
    # 判断是否在Jupyter环境中运行
    try:
        # 检查是否在Jupyter中运行
        get_ipython = globals().get('get_ipython', None)
        if get_ipython and 'IPKernelApp' in get_ipython().config:
            # 在Jupyter环境中运行，使用默认配置
            print("Running in Jupyter environment, using default configs")
            configs = get_default_configs()
        else:
            # 在命令行环境中运行，使用argparse
            configs = argparser()
    except:
        # 任何异常都使用argparse处理
        configs = argparser()
    
    # 设置实验名称
    if configs.name is None:
        configs.exp_name = \
            f'{os.path.basename(configs.model_name)}' + \
            f'{"_fp" if configs.freeze_pooler else ""}' + \
            f'_b{configs.batch_size}_e{configs.epochs}' + \
            f'_len{configs.max_length}_lr{configs.lr}'
    else:
        configs.exp_name = configs.name
    
    # 设置设备
    if configs.device is None:
        configs.device = torch.device(
            'cuda' if torch.cuda.is_available() else 'cpu'
        )
    
    # 调用训练函数
    train(configs)

Running in Jupyter environment, using default configs


/mnt/cfs/anaconda3/envs/pykt-hzw3.8/lib/python3.8/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1/20: 100%|██| 6/6 [00:01<00:00,  3.51batch/s, coarse_loss=0.564, fine_loss=0.614, loss=1.178]
/mnt/cfs/anaconda3/envs/pykt-hzw3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1760: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/mnt/cfs/anaconda3/envs/pykt-hzw3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1760: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. 


Epoch 1/20:
  Train Loss: 1.2734
  Train Coarse-grained Metrics:
    Micro F1: 42.86
    Macro F1: 22.20
    Accuracy: 17.71
  Train Fine-grained Metrics (Unconstrained):
    Micro F1: 17.51
    Macro F1: 11.99
    Accuracy: 0.00
  Train Fine-grained Metrics (Constrained):
    Micro F1: 15.28
    Macro F1: 8.92
    Accuracy: 0.00
  Val Loss: 1.0928
  Val Coarse-grained Metrics:
    Micro F1: 60.00
    Macro F1: 20.00
    Accuracy: 37.04
  Val Fine-grained Metrics (Unconstrained):
    Micro F1: 22.50
    Macro F1: 7.19
    Accuracy: 0.00
  Val Fine-grained Metrics (Constrained):
    Micro F1: 26.87
    Macro F1: 7.32
    Accuracy: 0.00


Epoch 2/20: 100%|██| 6/6 [00:00<00:00,  8.04batch/s, coarse_loss=0.452, fine_loss=0.495, loss=0.948]
/mnt/cfs/anaconda3/envs/pykt-hzw3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1760: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/mnt/cfs/anaconda3/envs/pykt-hzw3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1760: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/mnt/cfs/anaconda3/envs/pykt-hzw3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1760: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predi


Epoch 2/20:
  Train Loss: 1.0306
  Train Coarse-grained Metrics:
    Micro F1: 55.77
    Macro F1: 19.59
    Accuracy: 32.29
  Train Fine-grained Metrics (Unconstrained):
    Micro F1: 11.21
    Macro F1: 2.48
    Accuracy: 3.12
  Train Fine-grained Metrics (Constrained):
    Micro F1: 11.76
    Macro F1: 2.56
    Accuracy: 4.17
  Val Loss: 0.9171
  Val Coarse-grained Metrics:
    Micro F1: 62.30
    Macro F1: 20.65
    Accuracy: 40.74
  Val Fine-grained Metrics (Unconstrained):
    Micro F1: 0.00
    Macro F1: 0.00
    Accuracy: 0.00
  Val Fine-grained Metrics (Constrained):
    Micro F1: 0.00
    Macro F1: 0.00
    Accuracy: 0.00


...

Epoch 6/20: 100%|██| 6/6 [00:00<00:00,  8.07batch/s, coarse_loss=0.392, fine_loss=0.374, loss=0.765]
/mnt/cfs/anaconda3/envs/pykt-hzw3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1760: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/mnt/cfs/anaconda3/envs/pykt-hzw3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1760: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/mnt/cfs/anaconda3/envs/pykt-hzw3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1760: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predi


Epoch 6/20:
  Train Loss: 0.7452
  Train Coarse-grained Metrics:
    Micro F1: 60.67
    Macro F1: 23.55
    Accuracy: 40.62
  Train Fine-grained Metrics (Unconstrained):
    Micro F1: 1.20
    Macro F1: 0.46
    Accuracy: 2.08
  Train Fine-grained Metrics (Constrained):
    Micro F1: 1.20
    Macro F1: 0.46
    Accuracy: 2.08
  Val Loss: 0.7624
  Val Coarse-grained Metrics:
    Micro F1: 60.38
    Macro F1: 21.05
    Accuracy: 33.33
  Val Fine-grained Metrics (Unconstrained):
    Micro F1: 0.00
    Macro F1: 0.00
    Accuracy: 0.00
  Val Fine-grained Metrics (Constrained):
    Micro F1: 0.00
    Macro F1: 0.00
    Accuracy: 0.00
Early stopping triggered.

===== Testing best model =====


/mnt/cfs/anaconda3/envs/pykt-hzw3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1760: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/mnt/cfs/anaconda3/envs/pykt-hzw3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1760: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/mnt/cfs/anaconda3/envs/pykt-hzw3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1760: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor 


===== Final Test Results =====
Final micro f1: 26.87
Final macro f1: 7.32

Coarse-grained micro f1: 60.00
Fine-grained micro f1: 26.87

Coarse-grained macro f1: 20.00
Fine-grained macro f1: 7.32

Accuracy: 0.00


+---------------+---------------+---------------+---------------+---------------+---------------+
| Final         | Final         | Course-       | Fine-grained  | Course-       | Fine-grained  |
| micro f1      | macro f1      | grained micro f1 | micro f1      | grained macro f1 | macro f1      |
+---------------+---------------+---------------+---------------+---------------+---------------+
| 26.87         | 7.32          | 60.00         | 26.87         | 20.00         | 7.32          |
+---------------+---------------+---------------+---------------+---------------+---------------+


batch_coarse_loss,█▇▇▆▅▅▅▃▃▄▅▃▄▄▃▃▃▃▃▃▃▃▃▂▃▃▂▂▁▂▂▁▂▂▁▂
batch_fine_loss,██▇▇▆▆▆▅▅▄▄▄▃▃▃▃▂▃▂▂▁▃▁▂▂▂▂▁▂▂▁▁▂▂▁▂
batch_loss,██▇▆▆▆▅▄▄▄▅▃▃▄▃▃▃▃▃▂▂▃▂▂▂▂▂▂▂▂▂▁▂▂▁▂
epoch,▁▂▄▅▇█
final_macro_f1,▁
final_micro_f1,▁
test_coarse_accuracy,▁
test_coarse_macro_f1,▁
test_coarse_micro_f1,▁
test_constrained_fine_accuracy,▁
test_constrained_fine_macro_f1,▁
